In [22]:
# ML_in_Finance-Deep-Learning-Interaction
# Author: Matthew Dixon
# Version: 1.0 (08.09.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6 and Tensorflow 1.3.0
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F., I. Halperin and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 

In [3]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import math
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.regularizers import l1,l2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
import statsmodels.api as sm
import statsmodels.stats.diagnostic as tds
from statsmodels.api import add_constant
from scipy import stats

Using TensorFlow backend.


# Overview
The purpose of this notebook is to illustrate a neural network interpretability method which is compatible with linear regression, including an interaction term. In linear regression, provided the independent variables are scaled, one can view the regression coefficients as a measure of importance of the variables and their interaction effect. Equivalently, the dependent variable can be differentiated w.r.t. the inputs to give the coefficient, with the interaction obtained from the cross-term in the Hessian. Similarly, the derivatives of the network w.r.t. the inputs are a non-linear generalization of interpretability in a linear regression model with interaction effects. Moreover, we should expect the neural network gradients to approximate the regression model coefficients when the data is generated by a linear regression model with interaction terms. Various simple experimental tests, corresponding to Section 4 of Chpt 5, are performed to illustrate the properties of network interpretability.

# Simple Data Generation Process (DGP)

Generate data from a regression model with an interaction term 

$Y=X_1+X_2 + X_1X_2+\epsilon, ~X_1, X_2 \sim N(0,1,), \epsilon \sim N(0,\sigma_n^2)$

In [21]:
M = 5000
np.random.seed(7)
X = np.zeros(shape=(M,2))
sigma_n = 0.01
X[:int(M/2),0]= np.random.randn(int(M/2))
X[:int(M/2),1]= np.random.randn(int(M/2))
# use antithetic sampling to reduce sample bias in the mean
X[int(M/2):,0]= -X[:int(M/2),0]
X[int(M/2):,1]= -X[:int(M/2),1]

eps= np.random.randn(M)
Y= 1.0*X[:,0] + 1.0*X[:,1] + 1.0*X[:,0]*X[:,1] + sigma_n*eps.flatten()

# Use OLS to fit the model to the data

In [7]:
ols_results = sm.OLS(Y, sm.add_constant(X)).fit()

In [8]:
y_ols=ols_results.predict(sm.add_constant(X))

In [9]:
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     5052.
Date:                Tue, 12 May 2020   Prob (F-statistic):               0.00
Time:                        19:09:20   Log-Likelihood:                -7103.5
No. Observations:                5000   AIC:                         1.421e+04
Df Residuals:                    4997   BIC:                         1.423e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0243      0.014      1.713      0.087      -0.004       0.052
x1             0.9999      0.014     70.236      0.000       0.972       1.028
x2             1.0000      0.014     70.164      0.000       0.972       1.028
==============================================================================
Omnibus:                      846.889   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16614.377
Skew:                           0.168   Prob(JB):                         0.00
Kurtosis:                      11.924   Cond. No.                         1.02
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Compare with a ffwd neural network with no hidden layers

In [10]:
def linear_NN0_model(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(1, input_dim=2, kernel_initializer='normal')) #, activation='None'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [11]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

In [12]:
lm = KerasRegressor(build_fn=linear_NN0_model, epochs=40, batch_size=10, verbose=1, callbacks=[es])

In [13]:
lm.fit(X,Y)

Epoch 1/40
5000/5000 [==============================] - 1s 112us/step - loss: 2.1870 - mean_absolute_error: 1.0362 - mean_squared_error: 2.1870
Epoch 2/40
5000/5000 [==============================] - 0s 78us/step - loss: 1.4521 - mean_absolute_error: 0.7911 - mean_squared_error: 1.4521
Epoch 3/40
5000/5000 [==============================] - 0s 80us/step - loss: 1.1473 - mean_absolute_error: 0.6759 - mean_squared_error: 1.1473
Epoch 4/40
5000/5000 [==============================] - 0s 74us/step - loss: 1.0432 - mean_absolute_error: 0.6392 - mean_squared_error: 1.0432
Epoch 5/40
5000/5000 [==============================] - 0s 75us/step - loss: 1.0133 - mean_absolute_error: 0.6288 - mean_squared_error: 1.0133
Epoch 6/40
5000/5000 [==============================] - 0s 75us/step - loss: 1.0068 - mean_absolute_error: 0.6265 - mean_squared_error: 1.0068
Epoch 7/40
5000/5000 [==============================] - 0s 76us/step - loss: 1.0052 - mean_absolute_error: 0.6260 - mean_squared_error: 1.005

## Check that the weights are close to one

In [16]:
W=lm.model.layers[0].get_weights()[0]
b=lm.model.layers[0].get_weights()[1]
print("Weights: "+ str(W))
print("Bias: " +str(b))

Weights: [[0.99630827]
 [0.99237144]]
Bias: [0.02239072]


# Compare with a FFW Neural Network with one hidden layer (unactivated)

In [17]:
n = 10 # number of hidden units

In [18]:
def linear_NN1_model(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(n, input_dim=2, kernel_initializer='normal')) 
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [19]:
lm = KerasRegressor(build_fn=linear_NN1_model, epochs=50, batch_size=10, verbose=1, callbacks=[es])

In [20]:
lm.fit(X,Y)

Epoch 1/50
5000/5000 [==============================] - 1s 123us/step - loss: 1.6924 - mean_absolute_error: 0.8499 - mean_squared_error: 1.6924
Epoch 2/50
5000/5000 [==============================] - 0s 89us/step - loss: 1.0064 - mean_absolute_error: 0.6269 - mean_squared_error: 1.0064
Epoch 3/50
5000/5000 [==============================] - 0s 84us/step - loss: 1.0069 - mean_absolute_error: 0.6264 - mean_squared_error: 1.0069
Epoch 4/50
5000/5000 [==============================] - 0s 87us/step - loss: 1.0101 - mean_absolute_error: 0.6284 - mean_squared_error: 1.0101
Epoch 5/50
5000/5000 [==============================] - 0s 88us/step - loss: 1.0089 - mean_absolute_error: 0.6268 - mean_squared_error: 1.0089
Epoch 6/50
5000/5000 [==============================] - 0s 91us/step - loss: 1.0082 - mean_absolute_error: 0.6265 - mean_squared_error: 1.0082
Epoch 7/50
5000/5000 [==============================] - 0s 87us/step - loss: 1.0069 - mean_absolute_error: 0.6275 - mean_squared_error: 1.006

In [17]:
W1=lm.model.get_weights()[0]
b1=lm.model.get_weights()[1]
W2=lm.model.get_weights()[2]
b2=lm.model.get_weights()[3]
print(W1, W2)

[[-0.37814584  0.30338565 -0.27131096  0.3684254   0.33092746 -0.30008778
  -0.2542268  -0.29876462 -0.2947897   0.28479353]
 [-0.2638578   0.30114004 -0.3520979   0.26289687  0.20439926 -0.35611275
  -0.3233128  -0.2558721  -0.3140065   0.320782  ]] [[-0.28841314]
 [ 0.31222868]
 [-0.32610312]
 [ 0.30656126]
 [ 0.3128089 ]
 [-0.32401562]
 [-0.2921686 ]
 [-0.36572298]
 [-0.33868715]
 [ 0.3483182 ]]


## Check that the coefficients are close to one and the intercept is close to zero

In [18]:
beta_0=np.dot(np.transpose(W2), b1) + b2
beta_1=np.dot(np.transpose(W2), W1[0])
beta_2=np.dot(np.transpose(W2), W1[1])

In [19]:
print(beta_0,beta_1,beta_2)

[0.03241214] [0.9885408] [0.95098716]


# Compare with a FFW Neural Network with one hidden layer (tanh activated)

In [20]:
# number of hidden neurons
n=10

In [21]:
# with non-linear activation
def linear_NN1_model_act(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(n, input_dim=2, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [22]:
lm = KerasRegressor(build_fn=linear_NN1_model_act, epochs=100, batch_size=10, verbose=1, callbacks=[es])

In [23]:
lm.fit(X,Y)

Epoch 1/100
5000/5000 [==============================] - 1s 125us/step - loss: 2.7012 - mean_absolute_error: 1.2210 - mean_squared_error: 2.7012
Epoch 2/100
5000/5000 [==============================] - 0s 92us/step - loss: 1.9903 - mean_absolute_error: 1.0652 - mean_squared_error: 1.9903
Epoch 3/100
5000/5000 [==============================] - 0s 93us/step - loss: 1.9608 - mean_absolute_error: 1.0588 - mean_squared_error: 1.9608
Epoch 4/100
5000/5000 [==============================] - 0s 92us/step - loss: 1.9343 - mean_absolute_error: 1.0521 - mean_squared_error: 1.9343
Epoch 5/100
5000/5000 [==============================] - 0s 86us/step - loss: 1.9035 - mean_absolute_error: 1.0448 - mean_squared_error: 1.9035
Epoch 6/100
5000/5000 [==============================] - 0s 89us/step - loss: 1.8692 - mean_absolute_error: 1.0367 - mean_squared_error: 1.8692
Epoch 7/100
5000/5000 [==============================] - 0s 89us/step - loss: 1.8355 - mean_absolute_error: 1.0270 - mean_squared_error

## Compute the sensitivities

In [25]:
# Assume that the activation function is tanh
def sensitivities(lm, X):
    
    W1=lm.model.get_weights()[0]
    b1=lm.model.get_weights()[1]
    W2=lm.model.get_weights()[2]
    b2=lm.model.get_weights()[3]
    
    
    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta=np.array([0]*M*(p+1), dtype='float32').reshape(M,p+1)
    beta_interact=np.array([0]*M*p*p, dtype='float32').reshape(M,p,p)
    
    beta[:,0]= (np.dot(np.transpose(W2),np.tanh(b1)) + b2)[0] # intercept \beta_0= F_{W,b}(0)
    for i in range(M):
 
      Z1 = np.tanh(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1)
      #Z1 = np.maximum(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1,0) 
      
      D = np.diag(1-Z1**2) 
      D_prime =np.diag(-2*Z1*(1-Z1**2))   # needed for interaction term     
      #D = np.diag(np.sign(Z1))  
        
      for j in range(p):  
          beta[i,j+1]=np.dot(np.transpose(W2),np.dot(D,W1[j]))
          #interaction term 
          for k in range(p):
            beta_interact[i,j,k]=np.dot(np.transpose(W2),np.dot(np.diag(W1[j]), np.dot(D_prime,W1[k])))  
    
            
    return(beta, beta_interact)

In [26]:
beta, beta_inter=sensitivities(lm, X)

## Check that the intercept is close to one and the coefficients are close to one

In [27]:
print(np.mean(beta, axis=0))

[-0.06321367  1.0291194   0.9991432 ]


In [28]:
print(np.std(beta, axis=0))

[1.1101154e-06 9.6800876e-01 9.7116160e-01]


In [30]:
print(np.mean(beta_inter, axis=0)) # off-diagonals are interaction terms

[[-0.01137162  0.9574556 ]
 [ 0.9574556  -0.01866435]]
